In [1]:
#!/usr/bin/env python3
"""
Clean NDVI dataset, train RandomForest model for Option B, save artifacts.

Inputs:
 - /mnt/data/ndvi_filled_option_c_poly.xlsx

Outputs:
 - /mnt/data/ndvi_optionb_cleaned.csv
 - /mnt/data/rf_optionb_ndvi_model_v2.joblib
 - /mnt/data/optionb_results_summary.json
"""

import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# Paths (use uploaded /mnt/data)
# -------------------------
INPUT_PATH = "/content/ndvi_filled_option_c_poly.xlsx"
CLEAN_CSV = "/content/ndvi_optionb_cleaned.csv"
MODEL_PATH = "/content/rf_optionb_ndvi_model_v2.joblib"
RESULTS_JSON = "/content/optionb_results_summary.json"

# -------------------------
# Helper functions
# -------------------------
def load_data(path=INPUT_PATH):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Input file not found: {path}")
    df = pd.read_excel(path)
    df.columns = df.columns.str.lower().str.strip()
    return df

def clean_agronomic_years(df):
    # Ensure numeric where expected
    for col in ["planting_year", "harvest_year", "planting_month", "harvest_month"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # Fix inconsistent rows where planting_year > harvest_year
    mask = (df.get("planting_year").notna()) & (df.get("harvest_year").notna()) & (df["planting_year"] > df["harvest_year"])
    inconsistent = df[mask].copy()
    if not inconsistent.empty:
        # Strategy: set harvest_year = planting_year (align season) and keep note
        df.loc[mask, "harvest_year"] = df.loc[mask, "planting_year"]

    return df, inconsistent

def prepare_modeling_df(df):
    # Filter to potato product (case-insensitive)
    if "product" in df.columns:
        mask = df["product"].astype(str).str.lower().str.contains("potato", na=False)
        df = df.loc[mask].copy()
    else:
        # if no product column, assume dataset already filtered
        df = df.copy()

    # Keep important columns if present
    keep_cols = ["admin_1","admin_2","country","harvest_year","planting_year","planting_month","harvest_month",
                 "mean_annual_ndvi","area","production","yield"]
    existing = [c for c in keep_cols if c in df.columns]
    df = df[existing].copy()

    # Ensure numeric
    if "area" in df.columns:
        df["area"] = pd.to_numeric(df["area"], errors="coerce").fillna(0.0)
    if "mean_annual_ndvi" in df.columns:
        df["mean_annual_ndvi"] = pd.to_numeric(df["mean_annual_ndvi"], errors="coerce")

    # Drop rows missing core measurement columns
    required = []
    if "yield" in df.columns:
        required.append("yield")
    if "mean_annual_ndvi" in df.columns:
        required.append("mean_annual_ndvi")
    if required:
        df = df.dropna(subset=required).reset_index(drop=True)

    # Sort for time processing
    if "harvest_year" in df.columns:
        df = df.sort_values("harvest_year").reset_index(drop=True)
    return df

def add_lags_rolls_per_county(df, lags=(1,2,3)):
    # operate per county to maintain temporal integrity
    def _g(g):
        g = g.sort_values("harvest_year").reset_index(drop=True)
        for lag in lags:
            g[f"yield_lag_{lag}"] = g["yield"].shift(lag)
            g[f"ndvi_lag_{lag}"] = g["mean_annual_ndvi"].shift(lag)
        g["yield_roll3"] = g["yield"].rolling(3).mean().shift(1)
        g["ndvi_roll3"] = g["mean_annual_ndvi"].rolling(3).mean().shift(1)
        g["ndvi_change"] = g["mean_annual_ndvi"] - g["ndvi_lag_1"]
        # year index for temporal signal
        if "harvest_year" in g.columns:
            g["year_index"] = g["harvest_year"] - g["harvest_year"].min()
        else:
            g["year_index"] = np.arange(len(g))
        return g

    if "admin_1" in df.columns:
        df = df.groupby("admin_1", group_keys=False).apply(_g).reset_index(drop=True)
    else:
        df = _g(df)
    return df

def build_features(df, encode_county=True, max_dummies=100):
    # numeric core
    X_num = pd.DataFrame({
        "mean_annual_ndvi": df["mean_annual_ndvi"].astype(float),
        "area": df.get("area", pd.Series(0.0)).astype(float),
        "planting_month": df.get("planting_month", pd.Series(1)).fillna(1).astype(int),
        "year_index": df.get("year_index", pd.Series(0)).astype(int),
        "yield_lag_1": df.get("yield_lag_1"),
        "yield_lag_2": df.get("yield_lag_2"),
        "yield_lag_3": df.get("yield_lag_3"),
        "yield_roll3": df.get("yield_roll3"),
        "ndvi_lag_1": df.get("ndvi_lag_1"),
        "ndvi_lag_2": df.get("ndvi_lag_2"),
        "ndvi_lag_3": df.get("ndvi_lag_3"),
        "ndvi_roll3": df.get("ndvi_roll3"),
        "ndvi_change": df.get("ndvi_change")
    })

    X_num = X_num.astype(float).fillna(0.0)

    # county dummies
    if encode_county and "admin_1" in df.columns and df["admin_1"].nunique() <= max_dummies:
        dummies = pd.get_dummies(df["admin_1"].fillna("unknown"), prefix="adm1")
        X = pd.concat([X_num.reset_index(drop=True), dummies.reset_index(drop=True)], axis=1)
    else:
        X = X_num

    y = df["yield"].astype(float).values
    return X, y

def time_train_test_split(df, X, y, time_col="harvest_year"):
    # prefer time-based split: last harvest year -> test
    if time_col in df.columns:
        years = pd.to_numeric(df[time_col], errors="coerce").astype("Int64")
        last_year = int(years.max())
        train_mask = years < last_year
        test_mask = years == last_year

        # If too small, fall back to random split
        if train_mask.sum() < 10 or test_mask.sum() < 5:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            last_year = int(years.max())
        else:
            X_train = X.loc[train_mask]
            X_test = X.loc[test_mask]
            y_train = y[train_mask.values]
            y_test = y[test_mask.values]
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        last_year = None
    return X_train, X_test, y_train, y_test, last_year

def train_rf(X_train, y_train, do_tune=False):
    rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
    if not do_tune:
        rf.fit(X_train, y_train)
        return rf, None
    # tuning with TimeSeriesSplit
    param_dist = {
        "n_estimators": [200, 300, 500],
        "max_depth": [5, 10, 15, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    }
    tscv = TimeSeriesSplit(n_splits=5)
    rnd = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=20, cv=tscv, scoring="neg_mean_absolute_error", n_jobs=-1, random_state=42)
    rnd.fit(X_train, y_train)
    return rnd.best_estimator_, rnd

def evaluate(y_test, y_pred):
    if len(y_test) == 0:
        return {"mae": None, "rmse": None, "r2": None}
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = float(np.sqrt(mse))
    r2 = float(r2_score(y_test, y_pred))
    return {"mae": float(mae), "rmse": rmse, "r2": r2}

def compute_shap_summary(model, X, top_k=10):
    try:
        import shap
        explainer = shap.TreeExplainer(model)
        # sample for speed
        sample_n = min(500, X.shape[0])
        X_sample = X.sample(sample_n, random_state=42)
        shap_values = explainer.shap_values(X_sample)
        mean_abs = np.abs(shap_values).mean(axis=0)
        order_idx = np.argsort(-mean_abs)
        features = X_sample.columns[order_idx][:top_k].tolist()
        scores = mean_abs[order_idx][:top_k].tolist()
        shap_summary = [{"feature": f, "mean_abs_shap": float(s)} for f,s in zip(features, scores)]
        return shap_summary
    except Exception as e:
        return {"error": str(e)}

def compute_lime_summary(model, X_train, X_full, idx=None, num_features=8):
    try:
        from lime.lime_tabular import LimeTabularExplainer
        # pick representative instance
        if idx is None:
            idx = int(len(X_full) // 2)
        # ensure training sample small enough for LIME
        train_sample = X_train.sample(min(1000, X_train.shape[0]), random_state=1).values
        explainer = LimeTabularExplainer(train_sample, feature_names=X_train.columns.tolist(), mode='regression')
        exp = explainer.explain_instance(X_full.iloc[idx].values, model.predict, num_features=num_features)
        lime_list = exp.as_list()
        # split pos/neg
        pos = [t for t in lime_list if t[1] > 0]
        neg = [t for t in lime_list if t[1] < 0]
        lime_summary = {
            "instance_index": int(idx),
            "top_positive": pos[:5],
            "top_negative": neg[:5]
        }
        return lime_summary
    except Exception as e:
        return {"error": str(e)}

# -------------------------
# Recursive forecasting function (per-county) that updates lag features
# -------------------------
def recursive_forecast_by_county(agg_df, rf_model, features_list, years_ahead=1, area_override_val=None):
    """
    agg_df: aggregated per year DataFrame for a county with columns:
            ['harvest_year','area','production','mean_annual_ndvi','yield','admin_1']
    rf_model: trained model
    features_list: ORDERED list of features model expects
    years_ahead: int
    area_override_val: if provided, use same area for predictions
    """
    hist = agg_df.sort_values("harvest_year").reset_index(drop=True).copy()
    preds = []
    for step in range(years_ahead):
        last = hist.iloc[-1].copy()
        # compute lags
        for lag in [1,2,3]:
            if len(hist) >= lag:
                last[f"yield_lag_{lag}"] = hist["yield"].iloc[-lag]
                last[f"ndvi_lag_{lag}"] = hist["mean_annual_ndvi"].iloc[-lag]
            else:
                last[f"yield_lag_{lag}"] = hist["yield"].iloc[-1]
                last[f"ndvi_lag_{lag}"] = hist["mean_annual_ndvi"].iloc[-1]
        last["yield_roll3"] = hist["yield"].iloc[-3:].mean() if len(hist) >= 3 else hist["yield"].mean()
        last["ndvi_roll3"] = hist["mean_annual_ndvi"].iloc[-3:].mean() if len(hist) >= 3 else hist["mean_annual_ndvi"].mean()
        last["ndvi_change"] = last["mean_annual_ndvi"] - last.get("ndvi_lag_1", last["mean_annual_ndvi"])
        last["year_index"] = last["harvest_year"] - agg_df["harvest_year"].min() + (step + 1)
        # build feature vector in required order
        xrow = []
        for feat in features_list:
            if feat in last:
                xrow.append(last[feat])
            else:
                if feat.startswith("adm1_"):
                    xrow.append(1.0 if feat == f"adm1_{last['admin_1']}" else 0.0)
                else:
                    xrow.append(0.0)
        xarr = np.array([xrow], dtype=float)
        pred_y = float(rf_model.predict(xarr)[0])
        area_use = area_override_val if area_override_val is not None else last["area"]
        pred_prod = pred_y * area_use
        next_year = int(last["harvest_year"]) + 1
        preds.append({"harvest_year": next_year, "predicted_yield": float(pred_y), "predicted_production": float(pred_prod)})
        new_hist = {
            "harvest_year": next_year,
            "area": area_use,
            "production": pred_prod,
            "mean_annual_ndvi": last["mean_annual_ndvi"],  # keeping NDVI stable (optionally forecast NDVI separately)
            "yield": pred_y,
            "admin_1": last["admin_1"]
        }
        hist = pd.concat([hist, pd.DataFrame([new_hist])], ignore_index=True)
    return pd.DataFrame(preds)


# -------------------------
# Main
# -------------------------
def main():
    print("Loading dataset:", INPUT_PATH)
    df = load_data(INPUT_PATH)

    print("Cleaning agronomic years...")
    df_clean, inconsistent_rows = clean_agronomic_years(df)
    print("Inconsistent rows corrected:", int(inconsistent_rows.shape[0]))

    print("Saving cleaned CSV to:", CLEAN_CSV)
    df_clean.to_csv(CLEAN_CSV, index=False)

    print("Preparing modeling dataset...")
    mdl = prepare_modeling_df(df_clean)
    if mdl.shape[0] == 0:
        raise ValueError("No rows available for modeling after filtering. Check dataset columns and product filtering.")
    print("Modeling rows:", int(mdl.shape[0]))

    print("Adding lag & rolling features per county...")
    mdl = add_lags_rolls_per_county(mdl)

    # Drop rows missing the essential lag features (first few years per county)
    required_lags = ["yield_lag_1", "ndvi_lag_1"]
    mdl_model = mdl.dropna(subset=required_lags).reset_index(drop=True)
    print("After dropping rows lacking lags:", int(mdl_model.shape[0]))

    print("Building features and target...")
    X, y = build_features(mdl_model, encode_county=True, max_dummies=150)
    print("Feature matrix shape:", X.shape)

    print("Splitting train/test by time...")
    X_train, X_test, y_train, y_test, last_year = time_train_test_split(mdl_model, X, y)
    print("Train rows:", int(X_train.shape[0]), "Test rows:", int(X_test.shape[0]), "Last year used as test:", last_year)

    # Train model
    print("Training RandomForest (no hyperparameter tuning by default)...")
    rf, rnd = train_rf(X_train, y_train, do_tune=False)

    # Save model
    print("Saving model to:", MODEL_PATH)
    joblib.dump(rf, MODEL_PATH)

    # Evaluate
    y_pred = rf.predict(X_test) if X_test.shape[0] > 0 else np.array([])
    results = evaluate(y_test, y_pred)
    print("Evaluation metrics:", results)

    # Compute SHAP summary (lightweight)
    print("Computing SHAP summary (mean |SHAP| per feature)...")
    shap_summary = compute_shap_summary(rf, X)

    # Compute LIME summary for representative instance
    print("Computing LIME summary (representative instance)...")
    lime_summary = compute_lime_summary(rf, X_train, X)

    # Save final summary JSON
    summary = {
        "input_path": INPUT_PATH,
        "clean_csv": CLEAN_CSV,
        "model_path": MODEL_PATH,
        "n_rows_original": int(pd.read_excel(INPUT_PATH).shape[0]),
        "n_rows_cleaned": int(df_clean.shape[0]),
        "n_rows_modeling": int(mdl_model.shape[0]),
        "unique_counties": int(mdl_model["admin_1"].nunique()) if "admin_1" in mdl_model.columns else 0,
        "last_harvest_year_used_as_test": int(last_year) if last_year is not None else None,
        "train_rows": int(X_train.shape[0]),
        "test_rows": int(X_test.shape[0]),
        "evaluation": results,
        "features": X.columns.tolist(),
        "shap_summary": shap_summary,
        "lime_summary": lime_summary
    }

    with open(RESULTS_JSON, "w") as f:
        json.dump(summary, f, indent=2, default=lambda o: (o.__name__ if callable(o) else str(o)))
    print("Saved results summary to:", RESULTS_JSON)
    print("Done.")

if __name__ == "__main__":
    main()


Loading dataset: /content/ndvi_filled_option_c_poly.xlsx
Cleaning agronomic years...
Inconsistent rows corrected: 562
Saving cleaned CSV to: /content/ndvi_optionb_cleaned.csv
Preparing modeling dataset...
Modeling rows: 556
Adding lag & rolling features per county...
After dropping rows lacking lags: 524
Building features and target...
Feature matrix shape: (524, 45)
Splitting train/test by time...
Train rows: 495 Test rows: 29 Last year used as test: 2021
Training RandomForest (no hyperparameter tuning by default)...
Saving model to: /content/rf_optionb_ndvi_model_v2.joblib
Evaluation metrics: {'mae': 1.771960523571059, 'rmse': 2.4526329433850322, 'r2': 0.5484536395639259}
Computing SHAP summary (mean |SHAP| per feature)...
Computing LIME summary (representative instance)...
Saved results summary to: /content/optionb_results_summary.json
Done.
